In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [32]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB


In [4]:
from patient.simulator import Simulator
from patient.policy import *

## Assortment Linear Program

In [83]:
utilities = [random.random() for i in range(10)]
exit_option = random.random() 

In [84]:
best_assortment = []
best_score = 0
for assortment_combo in range(2**len(utilities)):
    assortment = [int(j) for j in bin(assortment_combo)[2:].zfill(len(utilities))]
    score = 0

    denom = np.sum([utilities[i]*assortment[i] for i in range(len(utilities))])+ exit_option
    for i in range(len(utilities)):
        prob = utilities[i]*assortment[i]/denom 
        score += prob*utilities[i] 
    if score > best_score:
        best_score = score 
        best_assortment = assortment
best_assortment, best_score

([0, 1, 1, 0, 1, 1, 1, 0, 0, 0], 0.6023784449512034)

In [85]:
N = len(utilities)
z_star = best_score 
u_coeffs = utilities 
model = gp.Model("ILP")
y = model.addVars(N, vtype=GRB.BINARY, name="y")
objective_expr = gp.LinExpr()
for j in range(N):
    objective_expr += (u_coeffs[j] - z_star) * u_coeffs[j] * y[j]/exit_option
model.setObjective(objective_expr, GRB.MAXIMIZE)
model.setParam('OutputFlag', 0)
for j in range(N):
    model.addConstr(y[j] >= 0)
    model.addConstr(y[j] <= 1)
model.optimize()
y = [int(y[j].x) for j in range(N)]
y

[0, 1, 1, 0, 1, 1, 1, 0, 0, 0]

In [86]:
model = gp.Model("LP")
model.setParam('OutputFlag', 0)
w = model.addVars(N+1, name="w")
objective_expr = gp.LinExpr()
for j in range(1, N+1):
    objective_expr += u_coeffs[j-1] * w[j]
model.setObjective(objective_expr, GRB.MAXIMIZE)

sum_expr = gp.LinExpr()
for j in range(1, N+1):
    sum_expr += w[j]
model.addConstr(sum_expr + w[0] == 1)

for j in range(1, N+1):
    if u_coeffs[j-1]>0:
        model.addConstr(w[j] / u_coeffs[j-1] <= w[0]/exit_option)
    else:
        model.addConstr(w[j]  <= 0)
model.optimize()

w_vals = [round((w[j].x*exit_option)/(u_coeffs[j-1]*w[0].x)) for j in range(1,N+1)]
w_vals

[0, 1, 1, 0, 1, 1, 1, 0, 0, 0]

## Patient-Provider Matches

In [5]:
num_patients = 20
num_providers = 10

In [6]:
s = Simulator(num_patients,num_providers)

In [7]:
seed_list = list(range(42,52))

In [8]:
results = s.simulate_no_renetry(random_policy,seed_list=seed_list)
np.mean(results), np.std(results)

(13.2, 1.4696938456699067)

In [9]:
results = s.simulate_no_renetry(greedy_policy,seed_list=seed_list)
np.mean(results), np.std(results)

(15.7, 1.004987562112089)

In [160]:
results = s.simulate_no_renetry(brute_force_discount,seed_list=seed_list)
np.mean(results), np.std(results)

(16.3, 0.9)

In [161]:
s = Simulator(num_patients,num_providers)
results = s.simulate_with_renetry(random_policy,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(16.4, 0.9165151389911679, 0.17, 0.44844174649557333)

In [162]:
s = Simulator(num_patients,num_providers)
results = s.simulate_with_renetry(greedy_policy,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(16.7, 1.5524174696260022, 0.1, 0.33166247903553997)

In [163]:
results = s.simulate_with_renetry(brute_force_discount,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(17.0, 1.5491933384829668, 0.105, 0.33760183648789593)

In [164]:
results = s.simulate_with_renetry(brute_force_discount_lamb,seed_list=seed_list)
np.mean(results['matches']), np.std(results['matches']), np.mean(results['waittimes']), np.std(results['waittimes'])

(17.3, 1.5524174696260025, 0.09, 0.31921779399024736)